In [50]:
from PIL import Image
import numpy as np
import os
from progressbar import progressbar
from shutil import copyfile

In [51]:
def add_margin(pil_img, top, right, bottom, left):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new('RGBA', (new_width, new_height), color='black')
    result.paste(pil_img.convert('RGBA'), (left, top))
    return result

def resize_width_keep_ratio(image, new_w):
    old_w = image.size[0]
    factor = new_w / old_w
    new_h = int(image.size[1] * factor)
    image = image.resize((new_w, new_h))
    return image

def compute_paddings(image, new_shape):
    pad = int(np.floor((new_shape - image.size[1])/2))
    bot_pad = pad
    if 2*pad + image.size[1] < new_shape:
        missing = new_shape - (2*pad + image.size[1])
        top_pad = pad + missing
        return pad + missing, pad
    return pad, pad
        



In [52]:
# im_new = add_margin(im, 50, 10, 0, 100, (0, 0, 0))

In [60]:
load_path = './data/FlagsMashupBot/raw_local/'
save_path = './data/FlagsMashupBot/square_png/'
os.makedirs(save_path, exist_ok=True)
files = [f for f in os.listdir(load_path) if not f.startswith('.')]
len(files)

2688

In [61]:
# im = Image.open(load_path + files[10])
# im.size

In [62]:
shapes = []
valid_files = []
for image_file in progressbar(files):
    try:
        im = Image.open(load_path + image_file)    
        shapes.append(im.size)
        valid_files.append(image_file)
    except:
        pass
    

100% (2688 of 2688) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


In [63]:
max_w = max([s[0] for s in shapes])
max_h = max([s[1] for s in shapes])
print(max_w, max_h)

4800 4000


In [64]:
new_shape = 256
im_format = 'png'
shapes = []
for image_file in progressbar(valid_files):
    im = Image.open(load_path + image_file)    
    im = resize_width_keep_ratio(im, new_shape)
    top_pad, bot_pad = compute_paddings(im, new_shape)
    im = add_margin(im, top_pad, 0, bot_pad, 0)
    im_name = image_file.split('.')[0]
    im_path_w_format = '{}{}.{}'.format(save_path, im_name, im_format)
    im.save(im_path_w_format)

100% (2688 of 2688) |####################| Elapsed Time: 0:09:06 Time:  0:09:06


### train_val_test split

In [65]:
save_path = './data/FlagsMashupBot/square_png/'
train_path = './data/FlagsMashupBot/autoencoder_images/'
square_files = [f for f in os.listdir(save_path) if not f.startswith('.')]

In [66]:
len(square_files)

6115

In [74]:
np.random.shuffle(square_files)

In [78]:
tr_cut = int(len(square_files) * 0.82)
# val_cut = int(len(square_files) * 0.8)

In [79]:
train_files = square_files[:tr_cut]
valid_files = square_files[tr_cut:]
# valid_files = square_files[tr_cut:val_cut]
# test_files = square_files[val_cut:]

In [80]:
# for dest, files in zip(['train', 'val', 'test'], [train_files, valid_files, test_files]):
for dest, files in zip(['train', 'val'], [train_files, valid_files]):
    path = '{}/{}/0/'.format(train_path, dest)
    os.makedirs(path, exist_ok=True)
    for f in files:
        src_file = save_path + '/' + f
        dest_file = path + f
#         print(src_file, dest_file)
        copyfile(src_file, dest_file)